In [ ]:
!pip install git+https://github.com/huggingface/diffusers.git

  Cloning https://github.com/huggingface/diffusers.git to /tmp/pip-req-build-9v8lt3a9
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/diffusers.git /tmp/pip-req-build-9v8lt3a9
  Resolved https://github.com/huggingface/diffusers.git to commit d90cd3621dc9aea168ec928ff3aa9b977eb49c20
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.33.0.dev0-py3-none-any.whl size=3350486 sha256=421711985f00aa06fe1911ea98064e9b9d73c8e89c4c8a567a6759a47959d687
  Stored in directory: /tmp/pip-ephem-wheel-cache-t1h639kk/wheels/d2/5c/5f/16639722ea17ecb73ab461b81718584bac08af2801619786b9
Successfully built diffusers
  Attempting uninstall: diffusers
    Found existing installation: diffusers 0.32.2
    Uninstalling diffusers-0.32.2:
      Successfully uninstalled diffusers-0.32.2


In [ ]:
!pip install -U bitsandbytes

In [ ]:
import torch
from diffusers import (
    BitsAndBytesConfig,
    HunyuanVideoTransformer3DModel,
    HunyuanVideoPipeline,
)
from diffusers.utils import export_to_video
from diffusers.hooks import apply_layerwise_casting
from transformers import LlamaModel

model_id = "hunyuanvideo-community/HunyuanVideo"
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True, bnb_4bit_compute_dtype=torch.bfloat16
)

text_encoder = LlamaModel.from_pretrained(model_id, subfolder="text_encoder", torch_dtype=torch.float16)
apply_layerwise_casting(text_encoder, storage_dtype=torch.float8_e4m3fn, compute_dtype=torch.float16)

# Apply 4-bit bitsandbytes quantization to Hunyuan DiT model
transformer = HunyuanVideoTransformer3DModel.from_pretrained(
    model_id,
    subfolder="transformer",
    quantization_config=quantization_config,
    torch_dtype=torch.bfloat16,
)

pipe = HunyuanVideoPipeline.from_pretrained(
    model_id, transformer=transformer, text_encoder=text_encoder, torch_dtype=torch.float16
)

# Enable memory saving
pipe.vae.enable_tiling()
pipe.enable_model_cpu_offload()

output = pipe(
    prompt="""Scene 1: A young boy, around 8-10 years old, stands in a well-lit indoor setting. He has short, neatly combed hair and wears casual clothes. On his head, a bold, uppercase letter "A" is clearly visible, either as a sticker, hat, or digital overlay. He looks forward with a neutral or slightly curious expression.

Scene 2: A Punjabi boy, dressed in a traditional kurta and wearing a colorful turban, stands confidently. On his head, the letter "B" is prominently placed, in the same style as in the previous scene. He has a cheerful or proud expression, adding a cultural touch to the scene.

Additional Notes: Smooth transitions between scenes, maintaining consistent color grading and lighting for a cohesive look.""",
    height=320,
    width=512,
    num_frames=61,
    num_inference_steps=30,
).frames[0]
export_to_video(output, "output.mp4", fps=15)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

(…)ion_pytorch_model.safetensors.index.json:   0%|          | 0.00/133k [00:00<?, ?B/s]

Fetching 6 files:   0%|          | 0/6 [00:00<?, ?it/s]

(…)pytorch_model-00006-of-00006.safetensors:   0%|          | 0.00/888M [00:00<?, ?B/s]

(…)pytorch_model-00005-of-00006.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

(…)pytorch_model-00001-of-00006.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

(…)pytorch_model-00004-of-00006.safetensors:   0%|          | 0.00/4.95G [00:00<?, ?B/s]

(…)pytorch_model-00002-of-00006.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

(…)pytorch_model-00003-of-00006.safetensors:   0%|          | 0.00/4.87G [00:00<?, ?B/s]

In [ ]:
# prompt: write code to display output.mp4 here

from IPython.display import HTML
from base64 import b64encode

def show_video(video_path):
  mp4 = open(video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=400 controls>
        <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

show_video('output.mp4')